In [1]:
from confluent_kafka import Producer
import json

from confluent_kafka import Consumer, KafkaError

topicName ='raw'
topicClean= 'clean-latest'
formatString ='%Y-%m-%dT%H:%M:%S.%f'


In [ ]:
#%pip install -U avro-python3

In [2]:
p = Producer({'bootstrap.servers': 'kafka-1:19092,kafka-2:19093'})


In [3]:
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered Timestamp:{timestamp} to {} [{}] key:{}'.format(msg.topic(), msg.partition(), msg.key(),timestamp=msg.timestamp()))


In [4]:
def getDatetimefromString(datestring):
    datetimeObject = datetime.strptime(datestring.replace('Z','000'), formatString)
    return datetimeObject

In [18]:
# test
#secondmessage= json.load('second_5b8449037c519100190fc72cTest.json')
#firstmessage= json.load('first_5b8449037c519100190fc72cTest-Copy1.json')

In [21]:
customercleanloader = Consumer({
    'bootstrap.servers': 'kafka-1:19092,kafka-2:19093',
    'group.id': 'cleanloader-consumer-group',
    'default.topic.config': {
        'auto.offset.reset': 'largest'
    }
})

customercleanloader.subscribe([topicClean])

In [22]:
lastMeasurement = None
lastRead =''
def SetLastDate(measurementarray):
    global lastMeasurement 
    global lastRead
    
    for m in measurementarray:
        if lastMeasurement is None:
            lastMeasurement = m
            lastRead = datetime.strptime(m['createdAt'].replace('Z','000'), formatString)
            continue
            
        dateTimestr = m['createdAt']
        measuredate = datetime.strptime(datetimeStr.replace('Z','000'), formatString)
        if( lastRead < measuredate) :
            lastMeasurement = m
            lastRead = measuredate
    


def loadCleanData():
    """
    loads the latest measurement time from the clean topic
    """
    go_on = True
    stopafter10 =10
    counter =0
    while go_on:
        counter+=1
        try:
            msg = customercleanloader.poll(1.0)

            if msg is None:
                continue
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    continue
                else:
                    print(msg.error())
                    break

           # print('Received message: {}'.format(msg.value().decode('utf-8')))
            SetLastDate(msg.value().decode('utf-8'))
            go_on = False
            if msg.value().decode('utf-8') == "STOP":
                go_on = False
        except KeyboardInterrupt:
            print("stopped by keyboard ")
            break
            
    customercleanloader.close()

In [12]:
#test
loadCleanData()

stopped by keyboard 


In [30]:
lastone = ""

def WritePM10(msg):
    key = msg.key().decode('utf-8')
    measurements = json.loads(msg.value().decode('utf-8'))
    
    onlyNew= True
    global lastone
    lastRead = lastone
    data=[]
    for m in measurements:      
        isNewValue =True
      #  isNewValue= not lastRead or (
      #      lastRead m['createdAt'] != 
      #      m['createdAt'])
        if(not onlyNew or isNewValue):
          #  printTemperatureFromBox(s)
            #print(m["value"])
            lastone = m                
   
            # Asynchronously produce a message, the delivery report callback
            # will be triggered from poll() above, or flush() below, when the message has
            # been successfully delivered or failed permanently.
            data.append(m)
            
    print("found {} new measurements".format(len(data)))
            
            
     # p.poll(0) 
  #  p.produce("clean-latest", key=box["_id"], value = data.encode('utf-8'), callback=delivery_report)

    # Wait for any outstanding messages to be delivered and delivery report
    # callbacks to be triggered.
   # p.flush()


In [31]:
cRaw = Consumer({
    'bootstrap.servers': 'kafka-1:19092,kafka-2:19093',
    'group.id': 'raw-to-clean-consumer-group',
    'default.topic.config': {
        'auto.offset.reset': 'largest'
    }
})

cRaw.subscribe([topicName])

#single run for testing
# todo fix avro schema
def SingleRun():
    go_on = True

    while go_on:
        msg = cRaw.poll(1.0)

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break

        print('Received message: {}'.format(msg.key().decode('utf-8')))
        WritePM10(msg)
        go_on = False
        if msg.value().decode('utf-8') == "STOP":
            go_on = False

    cRaw.close()

SingleRun()

Received message: 5d76badc953683001aa283f5
found 526 new measurements


In [ ]:


#go_on = True
#
#while go_on:
#    msg = c.poll(1.0)
#
#    if msg is None:
#        continue
#    if msg.error():
#        if msg.error().code() == KafkaError._PARTITION_EOF:
#            continue
#        else:
#            print(msg.error())
#            break
#
#   # print('Received message: {}'.format(msg.value().decode('utf-8')))
#    WritePM10(msg.value().decode('utf-8'))
#    if msg.value().decode('utf-8') == "STOP":
#        go_on = False
#        
#c.close()

In [ ]:
go_on = False